In [2]:
pip install pandas statsmodels prophet scikit-learn
pip install forecast croston tbats

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing,SimpleExpSmoothing, Holt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import STL
from scipy.interpolate import UnivariateSpline
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Read the CSV file into a DataFrame
df = pd.read_csv('AUINSA.csv')

# Show the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Show columns and their types
print(df.info())

# Convert the `Date` column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set the `Date` column as the index
df.set_index('Date', inplace=True)

# Print the length of the entire dataset
print(f"Length of the entire dataset: {len(df)}")

# Split the data into train and test sets
train_data = df[:-12]
test_data = df[-12:]

# Print the length of the train and test sets
print(f"Length of train data: {len(train_data)}")
print(f"Length of test data: {len(test_data)}")

# Define a function to evaluate the model
def evaluate_model(actual, predicted):
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = mean_absolute_percentage_error(actual, predicted)
    mae = mean_absolute_error(actual, predicted)
    print(f'RMSE: {rmse:.3f}')
    print(f'MAPE: {mape:.3f}')
    print(f'MAE: {mae:.3f}')

# 1. ARIMA Model
arima_model = ARIMA(train_data['Inventory'], order=(1, 1, 1))
arima_model_fit = arima_model.fit()
arima_predictions = arima_model_fit.forecast(steps=len(test_data))

# Evaluate ARIMA model
print("\nARIMA Model Evaluation:")
evaluate_model(test_data['Inventory'], arima_predictions)

# 2. Exponential Smoothing (ETS) Model - HoltWinters1 (Single Exponential Smoothing)
ets_model = SimpleExpSmoothing(train_data['Inventory'])
ets_model_fit = ets_model.fit()
ets_predictions = ets_model_fit.forecast(steps=len(test_data))

# Evaluate ETS model
print("\nETS Model Evaluation:")
evaluate_model(test_data['Inventory'], ets_predictions)

# 3. Holt's Linear Trend Model
try:
    holt_model = Holt(train_data['Inventory'])
    holt_model_fit = holt_model.fit()
    holt_predictions = holt_model_fit.forecast(steps=len(test_data))

    # Evaluate Holt's Linear Trend model
    print("\nHolt's Linear Trend Model Evaluation:")
    evaluate_model(test_data['Inventory'], holt_predictions)
except Exception as e:
    print(f"Error implementing Holt's Linear Trend Model: {e}")
    holt_predictions = None

# 4. Seasonal Trend decomposition using LOESS (STL)
stl = STL(train_data['Inventory'], seasonal=13)
res = stl.fit()
stl_predictions = res.trend[-12:] + res.seasonal[-12:]

# Evaluate STL decomposition model
print("\nSTL Decomposition Model Evaluation:")
evaluate_model(test_data['Inventory'], stl_predictions)

# 5. Holt-Winters' Seasonal Method
hw_model = ExponentialSmoothing(train_data['Inventory'], trend='add', seasonal='add', seasonal_periods=12)
hw_model_fit = hw_model.fit()
hw_predictions = hw_model_fit.forecast(steps=len(test_data))

# Evaluate Holt-Winters' Seasonal method
print("\nHolt-Winters' Seasonal Method Evaluation:")
evaluate_model(test_data['Inventory'], hw_predictions)

# 6. Seasonal ARIMA with exogenous variables (SARIMAX)
sarimax_model = SARIMAX(train_data['Inventory'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarimax_model_fit = sarimax_model.fit()
sarimax_predictions = sarimax_model_fit.forecast(steps=len(test_data))

# Evaluate SARIMAX model
print("\nSARIMAX Model Evaluation:")
evaluate_model(test_data['Inventory'], sarimax_predictions)

# 7. Naive Forecast
naive_predictions = train_data['Inventory'][-12:].values

# Evaluate Naive model
print("\nNaive Model Evaluation:")
evaluate_model(test_data['Inventory'], naive_predictions)

# 8. Spline Interpolation
x = np.arange(len(train_data))
y = train_data['Inventory'].values
spline = UnivariateSpline(x, y, k=3)
x_test = np.arange(len(train_data), len(train_data) + len(test_data))
spline_predictions = spline(x_test)

# Evaluate Spline model
print("\nSpline Model Evaluation:")
evaluate_model(test_data['Inventory'], spline_predictions)

# 9. Theta method
#try:
#    from forecast import thetaf
#    theta_model = thetaf(train_data['Inventory'], h=len(test_data))
#    theta_predictions = theta_model.values.flatten()

    # Evaluate Theta method
 #   print("\nTheta Method Evaluation:")
  #  evaluate_model(test_data['Inventory'], theta_predictions)
#except ImportError:
 #   print("The 'forecast' library is not installed. Theta method will be excluded from the ensemble.")
  #  theta_predictions = None

# 10. Croston method
#try:
#    from croston import croston
#    croston_model = croston.fit_croston(train_data['Inventory'], 12)
#    croston_predictions = croston.predict_croston(croston_model, 12)

    # Evaluate Croston method
  #  print("\nCroston Method Evaluation:")
   # evaluate_model(test_data['Inventory'], croston_predictions)
#except ImportError:
 #   print("The 'croston' library is not installed. Croston method will be excluded from the ensemble.")
  #  croston_predictions = None

# 11. TBATS method
#try:
#    from tbats import TBATS
 #   tbats_model = TBATS(seasonal_periods=[12])
  #  tbats_model_fit = tbats_model.fit(train_data['Inventory'])
   # tbats_predictions = tbats_model_fit.forecast(steps=len(test_data))

    # Evaluate TBATS method
   # print("\nTBATS Method Evaluation:")
    #evaluate_model(test_data['Inventory'], tbats_predictions)
#except ImportError:
 #   print("The 'tbats' library is not installed. TBATS method will be excluded from the ensemble.")
  #  tbats_predictions = None

# Ensemble: Simple Average
models_predictions = [
    pred for pred in [
        arima_predictions, ets_predictions, holt_predictions,
        stl_predictions, hw_predictions, sarimax_predictions,
        naive_predictions, spline_predictions 
        #theta_predictions,
        #croston_predictions, 
        #tbats_predictions
    ] if pred is not None
]

ensemble_predictions = np.mean(models_predictions, axis=0)

# Evaluate the ensemble
print("\nEnsemble Model Evaluation:")
evaluate_model(test_data['Inventory'], ensemble_predictions)

| Date       | Inventory   |
|:-----------|:------------|
| 2020-01-01 | 532.127     |
| 2020-02-01 | 532.959     |
| 2020-03-01 | 567.032     |
| 2020-04-01 | 543.154     |
| 2020-05-01 | 450.778     |
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       48 non-null     object 
 1   Inventory  48 non-null     float64
dtypes: float64(1), object(1)
memory usage: 896.0+ bytes
None
Length of the entire dataset: 48
Length of train data: 36
Length of test data: 12

ARIMA Model Evaluation:
RMSE: 65.525
MAPE: 0.251
MAE: 49.673

ETS Model Evaluation:
RMSE: 66.334
MAPE: 0.255
MAE: 50.498

Holt's Linear Trend Model Evaluation:
RMSE: 39.516
MAPE: 0.137
MAE: 27.813

STL Decomposition Model Evaluation:
RMSE: 79.866
MAPE: 0.371
MAE: 68.373

Holt-Winters' Seasonal Method Evaluation:
RMSE: 101.437
MAPE: 0.463
MAE: 85.608
RUNNING THE L-BFGS-B CODE

           

 This problem is unconstrained.



At iterate    5    f=  3.20839D+00    |proj g|=  3.76982D-03

At iterate   10    f=  3.20673D+00    |proj g|=  2.36993D-02

At iterate   15    f=  3.19293D+00    |proj g|=  4.18225D-02

At iterate   20    f=  3.18581D+00    |proj g|=  2.23722D-03

At iterate   25    f=  3.18240D+00    |proj g|=  1.01367D-02

At iterate   30    f=  3.18203D+00    |proj g|=  1.15216D-04

At iterate   35    f=  3.18202D+00    |proj g|=  2.10199D-03

At iterate   40    f=  3.18199D+00    |proj g|=  1.53570D-03

At iterate   45    f=  3.18196D+00    |proj g|=  4.95348D-04

At iterate   50    f=  3.18192D+00    |proj g|=  3.93162D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn